In [1]:
import sys
import torch
import numpy as np

In [2]:
sys.path.append('/envs')
sys.path.append('/models')
import envs.viper as vpr
import models.models as models
import models.dqn as dqn

In [3]:
import importlib
importlib.reload(vpr); importlib.reload(models); importlib.reload(dqn)

<module 'models.dqn' from 'h:\\Coding Projects\\Python Projects\\Deep-XIV-Rotation\\models\\dqn.py'>

In [4]:
viper_env = vpr.Viper(_sks = 798)

b123


In [5]:
viper_env.get_max_actions()

11

In [6]:
viper_env.get_state_shape()

8

In [7]:
state = viper_env.state()
state[None, :]

tensor([[ 0.0000,  2.4500, 10.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]])

In [8]:
state.unsqueeze(0)

tensor([[ 0.0000,  2.4500, 10.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]])

In [9]:
agent = dqn.DQN(viper_env)

Layer (type:depth-idx)                   Output Shape              Param #
DenseNet                                 [8, 11]                   --
├─Linear: 1-1                            [8, 2048]                 18,432
├─Linear: 1-2                            [8, 11]                   22,539
Total params: 40,971
Trainable params: 40,971
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.33
Input size (MB): 0.00
Forward/backward pass size (MB): 0.13
Params size (MB): 0.16
Estimated Total Size (MB): 0.30


In [10]:
torch.device('cpu')

device(type='cpu')

In [11]:
inputs = torch.rand(5, viper_env.get_state_shape())
print(inputs)
preds = agent.get_action(inputs)
print(preds)

tensor([[0.6770, 0.4181, 0.9794, 0.6353, 0.6594, 0.4733, 0.1152, 0.2208],
        [0.6669, 0.4212, 0.8459, 0.0290, 0.5295, 0.4094, 0.7452, 0.9959],
        [0.6254, 0.4951, 0.8700, 0.8714, 0.7652, 0.5708, 0.3957, 0.8350],
        [0.1614, 0.8443, 0.5942, 0.7357, 0.0706, 0.5353, 0.5070, 0.1524],
        [0.9077, 0.5620, 0.7096, 0.4708, 0.0613, 0.4117, 0.9513, 0.1385]])
tensor([0, 4, 0, 0, 6], dtype=torch.int32)


In [12]:
print(viper_env.step(1))
viper_env.state()

Taking action steel_fangs
(200.0, 198.64)


tensor([ 0.7000,  3.4400,  0.7000,  1.0000, 59.3000,  0.0000,  0.0000,  0.0000])

In [13]:
for i in range(25):
    print(viper_env.step(0))
    print(viper_env.state())

Taking action nothing
(-1.0, 0.0)
tensor([ 0.8000,  3.4400,  0.8000,  1.0000, 59.2000,  0.0000,  0.0000,  0.0000])
Taking action nothing
(-1.0, 0.0)
tensor([ 0.9000,  3.4400,  0.9000,  1.0000, 59.1000,  0.0000,  0.0000,  0.0000])
Taking action nothing
(-1.0, 0.0)
tensor([ 1.0000,  3.4400,  1.0000,  1.0000, 59.0000,  0.0000,  0.0000,  0.0000])
Taking action nothing
(-1.0, 0.0)
tensor([ 1.1000,  3.4400,  1.1000,  1.0000, 58.9000,  0.0000,  0.0000,  0.0000])
Taking action nothing
(-1.0, 0.0)
tensor([ 1.2000,  3.4400,  1.2000,  1.0000, 58.8000,  0.0000,  0.0000,  0.0000])
Taking action nothing
(-1.0, 0.0)
tensor([ 1.3000,  3.4400,  1.3000,  1.0000, 58.7000,  0.0000,  0.0000,  0.0000])
Taking action nothing
(-1.0, 0.0)
tensor([ 1.4000,  3.4400,  1.4000,  1.0000, 58.6000,  0.0000,  0.0000,  0.0000])
Taking action nothing
(-1.0, 0.0)
tensor([ 1.5000,  3.4400,  1.5000,  1.0000, 58.5000,  0.0000,  0.0000,  0.0000])
Taking action nothing
(-1.0, 0.0)
tensor([ 1.6000,  3.4400,  1.6000,  1.0000, 58

In [14]:
print(viper_env.step(3))
viper_env.state()

Taking action hunters_sting
(327.6, 345.78)


tensor([ 4.1400,  2.4500,  0.7000,  2.0000, 55.8600,  0.0000, 39.3000,  0.0000])

In [15]:
viper_env.compute_gcd(2.5, 10, 0)

2.5